In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\DeepFake Voice Recognition\\audio-deepfake-detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\DeepFake Voice Recognition\\audio-deepfake-detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class CallbackPreparationConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    model_checkpoint_path: Path
    early_stopping_monitor: str
    early_stopping_patience: int

In [6]:
from audioClassifier.constants import *
from audioClassifier.utils.common import open_yaml_file, create_directories

In [7]:
class ConfigManager:
    def __init__(self, config_file = CONFIG_PATH, params_file = PARAMS_PATH):
        self.config = open_yaml_file(config_file)
        self.params = open_yaml_file(params_file)
        
        create_directories([self.config.artifacts_root])
        
    def read_callback_prep_config(self) -> CallbackPreparationConfig:
        callback_prep = self.config.callback_preparation
        
        checkpoint_dir = os.path.dirname(callback_prep.model_checkpoint_path)
        create_directories([checkpoint_dir, self.config.callback_preparation.tensorboard_log_dir])
        
        callback_prep_config = CallbackPreparationConfig(
            root_dir = callback_prep.root_dir,
            tensorboard_log_dir = callback_prep.tensorboard_log_dir,
            model_checkpoint_path = callback_prep.model_checkpoint_path,
            early_stopping_monitor = self.params.early_stopping_monitor,
            early_stopping_patience = self.params.early_stopping_patience 
        )
        
        return callback_prep_config

In [8]:
import time
from tensorflow.keras import callbacks
from audioClassifier import logger

In [9]:
class CallbackPreparation:
    def __init__(self, config: CallbackPreparationConfig):
        self.config = config
        
    def _get_timestamped_tb_directory(self):
        timestamp = time.strftime("%Y-%m-%d--%H:%M:%S")
        return os.path.join(self.config.tensorboard_log_dir, f"run-{timestamp}")
    
    def prepare_callbacks(self):
        tensorboard_log_dir = self._get_timestamped_tb_directory()
        
        try:
            tensorboard_callback = callbacks.TensorBoard(log_dir = tensorboard_log_dir)
            checkpoint_callback = callbacks.ModelCheckpoint(filepath = self.config.model_checkpoint_path, 
                                                            save_best_only = True)
            early_stopping_callback = callbacks.EarlyStopping(monitor = self.config.early_stopping_monitor, 
                                                              patience = self.config.early_stopping_patience, 
                                                              restore_best_weights = True)
            return [tensorboard_callback, checkpoint_callback, early_stopping_callback]
        except Exception as e:
            logger.error(f"Error creating callbacks: {e}")
            raise e

In [10]:
try:
    config = ConfigManager()
    callback_preparation_config = config.read_callback_prep_config()
    callback_preparation = CallbackPreparation(config = callback_preparation_config)
    callback_preparation.prepare_callbacks()
except Exception as e:
    raise e

2024-07-24 11:42:04,812 - common.py - INFO - YAML file 'config\config.yaml' was loaded successfully.
2024-07-24 11:42:04,819 - common.py - INFO - YAML file 'params.yaml' was loaded successfully.
2024-07-24 11:42:04,820 - common.py - INFO - Directory 'artifacts' created successfully or already exists.
2024-07-24 11:42:04,822 - common.py - INFO - Directory 'artifacts/callback_preparation/checkpoint_dir' created successfully or already exists.
2024-07-24 11:42:04,822 - common.py - INFO - Directory 'artifacts/callback_preparation/tensorboard_log_dir' created successfully or already exists.
